In [1]:
from cassandra.cluster import Cluster
import pandas as pd

In [2]:
df = pd.read_csv('./elinventory.csv')
df

,sku,name,description,warehouse_num
0,'PH01','Iphone11','Liquid Retina HD display',101
1,'PH02','Iphone 12','Resolution of : 1792‑by‑828‑pixel resolution...,101
2,'LA01','HP pavilion','Windows 11 Home AMD Ryzen 5000 processors',102
3,'LA04','MacBook M1','M1 includes Apple’s most advanced GPU. It be...,103
4,'TA01','Galaxy Tab 7 Lite','Light and Portable Sturdy Metal Frame 8MP Re...,104
5,'TA02','Galaxy Tab S7 FE 5G','Large Screen S PEN Multi-device Connectivity',106
6,'TV01','One Plus TV','Smarter tv. Smarter Choice. Vision Comfort (...,101
7,'TV02','Samsung TV','The new Samsung Neo QLED QN90C delivers pris...,105


In [3]:
clstr= Cluster()
session=clstr.connect()

In [4]:
session.execute("CREATE KEYSPACE IF NOT EXISTS tri WITH REPLICATION= {'class':'SimpleStrategy','replication_factor':1}")

In [5]:
rows=session.execute("DESCRIBE tri.product")
for row in rows:
    print(row)

Row(keyspace_name='tri', type='table', name='product', create_statement="CREATE TABLE tri.product (\n    sku text PRIMARY KEY,\n    description text,\n    name text,\n    warehouse_num int\n) WITH additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';")


In [6]:
session.execute("""
CREATE TABLE IF NOT EXISTS tri.product (
    sku VARCHAR,
    name VARCHAR,
    description VARCHAR,
    warehouse_num INT,
    PRIMARY KEY(sku)
);
""")

In [7]:
for index, row in df.iterrows():
    print(f"sku = {row[0]}, name = {row[1]}, description = {row[2]}, warehouse_num = {row[3]}")



sku = 'PH01', name =  'Iphone11', description =  'Liquid Retina HD display', warehouse_num = 101
sku = 'PH02', name =  'Iphone 12', description =  'Resolution of : 1792‑by‑828‑pixel resolution at 326 ppi', warehouse_num = 101
sku = 'LA01', name =  'HP pavilion', description =  'Windows 11 Home AMD Ryzen 5000 processors', warehouse_num = 102
sku = 'LA04', name =  'MacBook M1', description =  'M1 includes Apple’s most advanced GPU. It benefits from years of analysis of Mac applications including everyday apps and challenging pro workloads', warehouse_num = 103
sku = 'TA01', name =  'Galaxy Tab 7 Lite', description =  'Light and Portable Sturdy Metal Frame 8MP Rear Camera', warehouse_num = 104
sku = 'TA02', name =  'Galaxy Tab S7 FE 5G', description =  'Large Screen S PEN Multi-device Connectivity', warehouse_num = 106
sku = 'TV01', name =  'One Plus TV', description =  'Smarter tv. Smarter Choice. Vision Comfort (TÜV Rheinland Certified) 24W Speakers with Dolby Audio', warehouse_num = 10

In [19]:
for index, row in df.iterrows():
    query = f"INSERT INTO tri.product (sku, name, description, warehouse_num) VALUES ({row[0].strip()}, {row[1].strip()}, {row[2].strip()}, {row[3]});"
    session.execute(query)

In [26]:
rows = session.execute("select (sku, name, description, warehouse_num) from tri.product")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


sku=TA02, name=Galaxy Tab S7 FE 5G, description=Large Screen S PEN Multi-device Connectivity, warehouse_num=106
sku=LA04, name=MacBook M1, description=M1 includes Apple’s most advanced GPU. It benefits from years of analysis of Mac applications including everyday apps and challenging pro workloads, warehouse_num=103
sku=TA01, name=Galaxy Tab 7 Lite, description=Light and Portable Sturdy Metal Frame 8MP Rear Camera, warehouse_num=104
sku=TV02, name=Samsung TV, description=The new Samsung Neo QLED QN90C delivers pristine contrast color and design  rewarding you at every glance, warehouse_num=105
sku=LA01, name=HP pavilion, description=Windows 11 Home AMD Ryzen 5000 processors, warehouse_num=102
sku=TV01, name=One Plus TV, description=Smarter tv. Smarter Choice. Vision Comfort (TÜV Rheinland Certified) 24W Speakers with Dolby Audio, warehouse_num=101
sku=PH01, name=Iphone11, description=Liquid Retina HD display, warehouse_num=101
sku=PH02, name=Iphone 12, description=Resolution of : 1792‑

In [28]:
rows= session.execute("select (sku,name,description,warehouse_num) from tri.product where warehouse_num=101 ALLOW Filtering")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


sku=TV01, name=One Plus TV, description=Smarter tv. Smarter Choice. Vision Comfort (TÜV Rheinland Certified) 24W Speakers with Dolby Audio, warehouse_num=101
sku=PH01, name=Iphone11, description=Liquid Retina HD display, warehouse_num=101
sku=PH02, name=Iphone 12, description=Resolution of : 1792‑by‑828‑pixel resolution at 326 ppi, warehouse_num=101
